In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")

Libraries imported successfully!
Pandas version: 2.3.1


In [3]:
print("EXTRACTING DATA FROM CSV FILES")

# Load all CSV files
appointments_df = pd.read_csv('data/appointments.csv')
billing_df = pd.read_csv('data/billing.csv')
doctors_df = pd.read_csv('data/doctors.csv')
patients_df = pd.read_csv('data/patients.csv')
treatments_df = pd.read_csv('data/treatments.csv')
department_df = pd.read_csv('data/department.csv')

print(f"\n✓ Appointments: {len(appointments_df)} records")
print(f"✓ Billing: {len(billing_df)} records")
print(f"✓ Doctors: {len(doctors_df)} records")
print(f"✓ Patients: {len(patients_df)} records")
print(f"✓ Treatments: {len(treatments_df)} records")
print(f"✓ Departments: {len(department_df)} records")

EXTRACTING DATA FROM CSV FILES

✓ Appointments: 200 records
✓ Billing: 200 records
✓ Doctors: 10 records
✓ Patients: 50 records
✓ Treatments: 200 records
✓ Departments: 10 records


In [5]:
# Inspection

print(appointments_df.head())
print(billing_df.head())
print(doctors_df.head())
print(patients_df.head())
print(treatments_df.head())
print(department_df.head())

  appointment_id patient_id doctor_id appointment_date appointment_time  \
0           A001       P034      D009       2023-08-09         15:15:00   
1           A002       P032      D004       2023-06-09         14:30:00   
2           A003       P048      D004       2023-06-28          8:00:00   
3           A004       P025      D006       2023-09-01          9:15:00   
4           A005       P040      D003       2023-07-06         12:45:00   

  reason_for_visit     status  
0          Therapy  Scheduled  
1          Therapy    No-show  
2     Consultation  Cancelled  
3     Consultation  Cancelled  
4        Emergency    No-show  
  bill_id patient_id treatment_id   bill_date   amount payment_method  \
0    B001       P034         T001  2023-08-09  3941.97      Insurance   
1    B002       P032         T002  2023-06-09  4158.44      Insurance   
2    B003       P048         T003  2023-06-28  3731.55      Insurance   
3    B004       P025         T004  2023-09-01  4799.86      Insur

In [6]:
print("DATA PROFILING & QUALITY CHECKS")

def profile_dataframe(df, name):
    """Profile a dataframe for quality issues"""
    print(f"\n--- {name} ---")
    print(f"Shape: {df.shape}")
    print(f"Missing values:\n{df.isnull().sum()[df.isnull().sum() > 0]}")
    print(f"Duplicate rows: {df.duplicated().sum()}")

# Profile each dataset
profile_dataframe(appointments_df, "Appointments")
profile_dataframe(billing_df, "Billing")
profile_dataframe(doctors_df, "Doctors")
profile_dataframe(patients_df, "Patients")
profile_dataframe(treatments_df, "Treatments")

DATA PROFILING & QUALITY CHECKS

--- Appointments ---
Shape: (200, 7)
Missing values:
Series([], dtype: int64)
Duplicate rows: 0

--- Billing ---
Shape: (200, 7)
Missing values:
Series([], dtype: int64)
Duplicate rows: 0

--- Doctors ---
Shape: (10, 8)
Missing values:
Series([], dtype: int64)
Duplicate rows: 0

--- Patients ---
Shape: (50, 11)
Missing values:
Series([], dtype: int64)
Duplicate rows: 0

--- Treatments ---
Shape: (200, 6)
Missing values:
Series([], dtype: int64)
Duplicate rows: 0


In [7]:
print("\n--- Creating DimPatient ---")

# Create a copy to work with
dim_patient = patients_df.copy()

# Add surrogate key
dim_patient['PatientKey'] = range(1, len(dim_patient) + 1)

# Create FullName by combining first and last name
dim_patient['FullName'] = dim_patient['first_name'] + ' ' + dim_patient['last_name']

# Calculate Age and AgeGroup from date_of_birth
dim_patient['date_of_birth'] = pd.to_datetime(dim_patient['date_of_birth'])
dim_patient['Age'] = ((pd.Timestamp.now() - dim_patient['date_of_birth']).dt.days / 365).astype(int)

# Create AgeGroup categories
def categorize_age(age):
    """Categorize age into groups"""
    if age < 18:
        return 'Child'
    elif age < 35:
        return 'Young Adult'
    elif age < 50:
        return 'Adult'
    elif age < 65:
        return 'Senior'
    else:
        return 'Elderly'

dim_patient['AgeGroup'] = dim_patient['Age'].apply(categorize_age)

# Extract City from address (using street name as neighborhood/area)
# Since addresses are like "123 Elm St", we'll use the street name as area
dim_patient['City'] = dim_patient['address'].str.split().str[-2] + ' ' + dim_patient['address'].str.split().str[-1]

# Convert registration_date to proper datetime
dim_patient['RegistrationDate'] = pd.to_datetime(dim_patient['registration_date'])

# Select and rename columns for final dimension table
dim_patient = dim_patient[[
    'PatientKey',           # Surrogate key
    'patient_id',           # Natural key
    'FullName',             # Derived
    'first_name',           # Original
    'last_name',            # Original
    'gender',               # Original
    'date_of_birth',        # Original
    'Age',                  # Calculated
    'AgeGroup',             # Derived
    'City',                 # Derived from address
    'address',              # Original
    'contact_number',       # Original
    'email',                # Original
    'insurance_provider',   # Original
    'insurance_number',     # Original
    'RegistrationDate'      # Converted
]]

# Rename columns to match dimension naming convention
dim_patient.columns = [
    'PatientKey', 'PatientID', 'FullName', 'FirstName', 'LastName',
    'Gender', 'DateOfBirth', 'Age', 'AgeGroup', 'City', 'Address',
    'ContactNumber', 'Email', 'InsuranceProvider', 'InsuranceNumber',
    'RegistrationDate'
]

print(f"✓ DimPatient created: {len(dim_patient)} records")
print(f"  Age groups distribution:\n{dim_patient['AgeGroup'].value_counts()}")


--- Creating DimPatient ---
✓ DimPatient created: 50 records
  Age groups distribution:
AgeGroup
Young Adult    18
Adult          13
Senior         12
Elderly         7
Name: count, dtype: int64


In [8]:
print("\n--- Creating DimDepartment ---")

# For Option 1: We'll use only the 3 departments that match our doctor specializations
# Mapping: Pediatrics (2), Oncology (8), Dermatology (9)

# Map specializations to department IDs
specialization_to_dept = {
    'Pediatrics': 2,
    'Oncology': 8,
    'Dermatology': 9
}

# Create DimDepartment from the department.csv
dim_department = department_df.copy()
dim_department['DepartmentKey'] = range(1, len(dim_department) + 1)

# Filter to only include departments we have doctors for
used_dept_ids = list(specialization_to_dept.values())
dim_department = dim_department[dim_department['DepartmentID'].isin(used_dept_ids)].reset_index(drop=True)

# Re-assign surrogate keys after filtering
dim_department['DepartmentKey'] = range(1, len(dim_department) + 1)

print(f"✓ DimDepartment created: {len(dim_department)} records")
print(f"  Departments: {dim_department['DepartmentName'].tolist()}")



--- Creating DimDepartment ---
✓ DimDepartment created: 3 records
  Departments: ['Pediatrics', 'Oncology', 'Dermatology']


In [9]:
print("\n--- Creating DimDoctor ---")

dim_doctor = doctors_df.copy()

# Add surrogate key
dim_doctor['DoctorKey'] = range(1, len(dim_doctor) + 1)

# Create FullName
dim_doctor['FullName'] = dim_doctor['first_name'] + ' ' + dim_doctor['last_name']

# Map specialization to DepartmentID
dim_doctor['DepartmentID'] = dim_doctor['specialization'].map(specialization_to_dept)

# Select and rename columns
dim_doctor = dim_doctor[[
    'DoctorKey',
    'doctor_id',
    'FullName',
    'first_name',
    'last_name',
    'specialization',
    'DepartmentID',
    'phone_number',
    'years_experience',
    'hospital_branch',
    'email'
]]

dim_doctor.columns = [
    'DoctorKey', 'DoctorID', 'FullName', 'FirstName', 'LastName',
    'Specialization', 'DepartmentID', 'PhoneNumber', 'YearsExperience',
    'HospitalBranch', 'Email'
]

print(f"✓ DimDoctor created: {len(dim_doctor)} records")
print(f"  Specialization distribution:\n{dim_doctor['Specialization'].value_counts()}")


--- Creating DimDoctor ---
✓ DimDoctor created: 10 records
  Specialization distribution:
Specialization
Pediatrics     5
Dermatology    3
Oncology       2
Name: count, dtype: int64


In [10]:
print("\n--- Creating DimTreatment ---")

# Create unique treatment types with descriptions
treatment_types = treatments_df[['treatment_type', 'description']].drop_duplicates().reset_index(drop=True)

dim_treatment = treatment_types.copy()
dim_treatment['TreatmentKey'] = range(1, len(dim_treatment) + 1)

# Calculate average cost per treatment type
avg_costs = treatments_df.groupby('treatment_type')['cost'].mean().reset_index()
avg_costs.columns = ['treatment_type', 'AvgCost']

# Merge average costs
dim_treatment = dim_treatment.merge(avg_costs, on='treatment_type', how='left')

# Rename columns
dim_treatment.columns = ['TreatmentType', 'Description', 'TreatmentKey', 'AvgCost']
dim_treatment = dim_treatment[['TreatmentKey', 'TreatmentType', 'Description', 'AvgCost']]

print(f"✓ DimTreatment created: {len(dim_treatment)} records")
print(f"  Treatment types: {dim_treatment['TreatmentType'].tolist()}")



--- Creating DimTreatment ---
✓ DimTreatment created: 15 records
  Treatment types: ['Chemotherapy', 'MRI', 'MRI', 'MRI', 'ECG', 'Chemotherapy', 'Chemotherapy', 'Physiotherapy', 'Physiotherapy', 'ECG', 'ECG', 'X-Ray', 'X-Ray', 'Physiotherapy', 'X-Ray']


In [11]:
print("\n--- Creating DimDate ---")

# Get min and max dates from appointments to create date range
min_date = pd.to_datetime(appointments_df['appointment_date']).min()
max_date = pd.to_datetime(appointments_df['appointment_date']).max()

# Create date range
date_range = pd.date_range(start=min_date, end=max_date, freq='D')

# Create date dimension with all attributes
dim_date = pd.DataFrame({
    'FullDate': date_range
})

# Create DateKey in YYYYMMDD format
dim_date['DateKey'] = dim_date['FullDate'].dt.strftime('%Y%m%d').astype(int)

# Extract date components
dim_date['Year'] = dim_date['FullDate'].dt.year
dim_date['Quarter'] = dim_date['FullDate'].dt.quarter
dim_date['Month'] = dim_date['FullDate'].dt.month
dim_date['MonthName'] = dim_date['FullDate'].dt.strftime('%B')
dim_date['Day'] = dim_date['FullDate'].dt.day
dim_date['DayOfWeek'] = dim_date['FullDate'].dt.dayofweek + 1  # 1=Monday, 7=Sunday
dim_date['DayName'] = dim_date['FullDate'].dt.strftime('%A')
dim_date['WeekOfYear'] = dim_date['FullDate'].dt.isocalendar().week
dim_date['IsWeekend'] = dim_date['DayOfWeek'].isin([6, 7]).astype(int)

# Reorder columns
dim_date = dim_date[[
    'DateKey', 'FullDate', 'Year', 'Quarter', 'Month', 'MonthName',
    'Day', 'DayOfWeek', 'DayName', 'WeekOfYear', 'IsWeekend'
]]

print(f"✓ DimDate created: {len(dim_date)} records")
print(f"  Date range: {min_date.date()} to {max_date.date()}")



--- Creating DimDate ---
✓ DimDate created: 364 records
  Date range: 2023-01-01 to 2023-12-30


In [41]:
# Creating the fact table

print("CREATING FACT TABLE - FACT_VISITS")

# Start with appointments as the base for visits
fact_visits = appointments_df.copy()

# Convert dates to proper datetime
fact_visits['appointment_date'] = pd.to_datetime(fact_visits['appointment_date'])

# Create DateKey for joining with DimDate
fact_visits['DateKey'] = fact_visits['appointment_date'].dt.strftime('%Y%m%d').astype(int)


CREATING FACT TABLE - FACT_VISITS


In [42]:
print("\n--- Joining with dimension tables ---")

# Join with DimPatient to get PatientKey
fact_visits = fact_visits.merge(
    dim_patient[['PatientID', 'PatientKey']], 
    left_on='patient_id', 
    right_on='PatientID', 
    how='left'
)

# Join with DimDoctor to get DoctorKey
fact_visits = fact_visits.merge(
    dim_doctor[['DoctorID', 'DoctorKey', 'DepartmentID']], 
    left_on='doctor_id', 
    right_on='DoctorID', 
    how='left'
)

# Join with DimDepartment to get DepartmentKey
dim_department_lookup = dim_department[['DepartmentID', 'DepartmentKey']]
fact_visits = fact_visits.merge(
    dim_department_lookup,
    on='DepartmentID',
    how='left'
)

# Join with treatments to get treatment details
fact_visits = fact_visits.merge(
    treatments_df[['appointment_id', 'treatment_id', 'treatment_type', 'description', 'cost', 'treatment_date']],
    on='appointment_id',
    how='left'
)

fact_visits = fact_visits.merge(
    dim_treatment[['TreatmentType', 'Description', 'TreatmentKey']],
    left_on=['treatment_type', 'description'],
    right_on=['TreatmentType', 'Description'],
    how='left'
)

fact_visits = fact_visits.merge(
    billing_df[['treatment_id', 'amount', 'payment_method', 'payment_status']],
    on='treatment_id',
    how='left',
    suffixes=('', '_billing')
)

print("✓ All joins completed")


--- Joining with dimension tables ---
✓ All joins completed


In [43]:
print("\n--- Creating final fact table structure ---")

# Create VisitKey (surrogate key for fact table)
fact_visits['VisitKey'] = range(1, len(fact_visits) + 1)

# Map appointment status to visit outcome
status_mapping = {
    'Completed': 'Completed',
    'Scheduled': 'Scheduled',
    'Cancelled': 'Cancelled',
    'No-show': 'No-show'
}
fact_visits['VisitStatus'] = fact_visits['status'].map(status_mapping)

# Use reason_for_visit as DiagnosisCategory (proxy for diagnosis)
fact_visits['DiagnosisCategory'] = fact_visits['reason_for_visit']

# Handle missing or null values in measures
fact_visits['TreatmentCost'] = fact_visits['cost'].fillna(0)
fact_visits['BillAmount'] = fact_visits['amount'].fillna(0)
fact_visits['PaymentStatus'] = fact_visits['payment_status'].fillna('Unknown')

# Select final columns for fact table
fact_visits_final = fact_visits[[
    'VisitKey',              # Surrogate key
    'DateKey',               # FK to DimDate
    'PatientKey',            # FK to DimPatient
    'DoctorKey',             # FK to DimDoctor
    'DepartmentKey',         # FK to DimDepartment
    'TreatmentKey',          # FK to DimTreatment
    'appointment_id',        # Business key
    'VisitStatus',           # Degenerate dimension
    'DiagnosisCategory',     # Degenerate dimension (reason_for_visit)
    'TreatmentCost',         # Measure
    'BillAmount',            # Measure
    'PaymentStatus'          # Degenerate dimension
]]

# Rename columns to match fact table convention
fact_visits_final.columns = [
    'VisitKey', 'DateKey', 'PatientKey', 'DoctorKey', 'DepartmentKey',
    'TreatmentKey', 'AppointmentID', 'VisitStatus', 'DiagnosisCategory',
    'TreatmentCost', 'BillAmount', 'PaymentStatus'
]

print(f"✓ FactVisits created: {len(fact_visits_final)} records")



--- Creating final fact table structure ---
✓ FactVisits created: 200 records


In [44]:
# Data Quality Validation

print("\n--- Checking for orphan records (referential integrity) ---")

# Check for null foreign keys in fact table
null_checks = {
    'Null DateKey': fact_visits_final['DateKey'].isnull().sum(),
    'Null PatientKey': fact_visits_final['PatientKey'].isnull().sum(),
    'Null DoctorKey': fact_visits_final['DoctorKey'].isnull().sum(),
    'Null DepartmentKey': fact_visits_final['DepartmentKey'].isnull().sum(),
    'Null TreatmentKey': fact_visits_final['TreatmentKey'].isnull().sum()
}

for check, count in null_checks.items():
    status = "✓" if count == 0 else "✗"
    print(f"{status} {check}: {count}")

# Show summary statistics for measures
print("\n--- Measure Statistics ---")
print(fact_visits_final[['TreatmentCost', 'BillAmount']].describe())


--- Checking for orphan records (referential integrity) ---
✓ Null DateKey: 0
✓ Null PatientKey: 0
✓ Null DoctorKey: 0
✓ Null DepartmentKey: 0
✓ Null TreatmentKey: 0

--- Measure Statistics ---
       TreatmentCost   BillAmount
count     200.000000   200.000000
mean     2756.249250  2756.249250
std      1298.125308  1298.125308
min       534.030000   534.030000
25%      1563.412500  1563.412500
50%      2828.165000  2828.165000
75%      3836.627500  3836.627500
max      4973.630000  4973.630000


In [45]:
# Saving the Transformed Data to CSV

import os
output_dir = 'transformed_data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"✓ Created directory: {output_dir}")

# Save all dimension and fact tables
dim_patient.to_csv(f'{output_dir}/DimPatient.csv', index=False)
dim_doctor.to_csv(f'{output_dir}/DimDoctor.csv', index=False)
dim_department.to_csv(f'{output_dir}/DimDepartment.csv', index=False)
dim_treatment.to_csv(f'{output_dir}/DimTreatment.csv', index=False)
dim_date.to_csv(f'{output_dir}/DimDate.csv', index=False)
fact_visits_final.to_csv(f'{output_dir}/FactVisits.csv', index=False)

print(f"\n✓ DimPatient.csv - {len(dim_patient)} records")
print(f"✓ DimDoctor.csv - {len(dim_doctor)} records")
print(f"✓ DimDepartment.csv - {len(dim_department)} records")
print(f"✓ DimTreatment.csv - {len(dim_treatment)} records")
print(f"✓ DimDate.csv - {len(dim_date)} records")
print(f"✓ FactVisits.csv - {len(fact_visits_final)} records")


✓ Created directory: transformed_data

✓ DimPatient.csv - 50 records
✓ DimDoctor.csv - 10 records
✓ DimDepartment.csv - 3 records
✓ DimTreatment.csv - 15 records
✓ DimDate.csv - 364 records
✓ FactVisits.csv - 200 records


In [46]:
print("\n" + "="*70)
print("SAMPLE DATA PREVIEW")
print("="*70)

print("\n--- DimPatient (First 3 Records) ---")
print(dim_patient.head(3).to_string())

print("\n--- DimDoctor (First 3 Records) ---")
print(dim_doctor.head(3).to_string())

print("\n--- DimDepartment (All Records) ---")
print(dim_department.to_string())

print("\n--- DimTreatment (First 5 Records) ---")
print(dim_treatment.head(5).to_string())

print("\n--- DimDate (First 5 Records) ---")
print(dim_date.head(5).to_string())

print("\n--- FactVisits (First 5 Records) ---")
print(fact_visits_final.head(5).to_string())


SAMPLE DATA PREVIEW

--- DimPatient (First 3 Records) ---
   PatientKey PatientID        FullName FirstName  LastName Gender DateOfBirth  Age AgeGroup      City       Address  ContactNumber                    Email InsuranceProvider InsuranceNumber RegistrationDate
0           1      P001  David Williams     David  Williams      F  1955-06-04   70  Elderly   Pine Rd   789 Pine Rd     6939585183  david.williams@mail.com      WellnessCorp       INS840674       2022-06-23
1           2      P002     Emily Smith     Emily     Smith      F  1984-10-12   41    Adult  Maple Dr  321 Maple Dr     8228188767     emily.smith@mail.com       PulseSecure       INS354079       2022-01-15
2           3      P003     Laura Jones     Laura     Jones      M  1977-08-21   48    Adult  Maple Dr  321 Maple Dr     8397029847     laura.jones@mail.com       PulseSecure       INS650929       2022-02-07

--- DimDoctor (First 3 Records) ---
   DoctorKey DoctorID      FullName FirstName LastName Specialization  D

In [47]:
print("\n" + "="*70)
print("TRANSFORMATION SUMMARY REPORT")
print("="*70)

summary = f"""
EXTRACT:
--------
✓ Loaded 6 CSV files successfully
✓ Total raw records: {len(appointments_df) + len(billing_df) + len(doctors_df) + len(patients_df) + len(treatments_df) + len(department_df)}

TRANSFORM:
----------
✓ Created 5 dimension tables
✓ Created 1 fact table
✓ Added surrogate keys to all dimensions
✓ Calculated derived attributes (Age, AgeGroup, FullName, City)
✓ Created date dimension with {len(dim_date)} dates
✓ Joined {len(fact_visits_final)} fact records with all dimensions

DATA QUALITY:
-------------
✓ No missing foreign keys in fact table
✓ {len(fact_visits_final)} complete visit records
✓ Date range: {dim_date['FullDate'].min().date()} to {dim_date['FullDate'].max().date()}
✓ {len(dim_patient)} patients across {dim_patient['AgeGroup'].nunique()} age groups
✓ {len(dim_doctor)} doctors in {len(dim_department)} departments
✓ {len(dim_treatment)} treatment types

STAR SCHEMA:
------------
Fact Table: FactVisits ({len(fact_visits_final)} records)
  └── DimDate ({len(dim_date)} records)
  └── DimPatient ({len(dim_patient)} records)
  └── DimDoctor ({len(dim_doctor)} records)
  └── DimDepartment ({len(dim_department)} records)
  └── DimTreatment ({len(dim_treatment)} records)

READY FOR LOADING:
------------------
✓ All transformed files saved to: {output_dir}/
✓ Next step: Load to PostgreSQL using SQLAlchemy
"""

print(summary)

print("\n" + "="*70)
print("TRANSFORMATION COMPLETE! ✓")
print("="*70)


TRANSFORMATION SUMMARY REPORT

EXTRACT:
--------
✓ Loaded 6 CSV files successfully
✓ Total raw records: 670

TRANSFORM:
----------
✓ Created 5 dimension tables
✓ Created 1 fact table
✓ Added surrogate keys to all dimensions
✓ Calculated derived attributes (Age, AgeGroup, FullName, City)
✓ Created date dimension with 364 dates
✓ Joined 200 fact records with all dimensions

DATA QUALITY:
-------------
✓ No missing foreign keys in fact table
✓ 200 complete visit records
✓ Date range: 2023-01-01 to 2023-12-30
✓ 50 patients across 4 age groups
✓ 10 doctors in 3 departments
✓ 15 treatment types

STAR SCHEMA:
------------
Fact Table: FactVisits (200 records)
  └── DimDate (364 records)
  └── DimPatient (50 records)
  └── DimDoctor (10 records)
  └── DimDepartment (3 records)
  └── DimTreatment (15 records)

READY FOR LOADING:
------------------
✓ All transformed files saved to: transformed_data/
✓ Next step: Load to PostgreSQL using SQLAlchemy


TRANSFORMATION COMPLETE! ✓
